In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
directors_df = pd.read_csv('../data/train/directors.tsv', sep='\t') # 감독 정보
genres_df = pd.read_csv('../data/train/genres.tsv', sep='\t') # 장르 정보
titles_df = pd.read_csv('../data/train/titles.tsv', sep='\t') # 영화 제목 정보
writers_df = pd.read_csv('../data/train/writers.tsv', sep='\t') # 각본가 정보
years_df = pd.read_csv('../data/train/years.tsv', sep='\t') # 개봉 연도 정보

train_df = pd.read_csv('../data/train/train_ratings.csv') # 전체 학습 데이터

# directors_df EDA

In [ ]:
# 기본 정보 확인
print(directors_df.info())
print(directors_df.head())

# 결측치 확인
print(directors_df.isnull().sum())

# 중복 행 확인
print(directors_df[directors_df.duplicated()])

'directors_df'에는 결측치가 없다.

In [ ]:
# 고유 감독 수
unique_directors = directors_df['director'].nunique()
print(f"Unique directors: {unique_directors}")

# 감독별 빈도
director_counts = directors_df['director'].value_counts()
print(director_counts.head(10))  # 상위 10명

In [ ]:
# 감독 별 영화 수 계산
movie_count_per_director = directors_df['director'].value_counts()  # 감독별 참여한 영화 수

# 영화 수의 분포 계산
movie_count_distribution = movie_count_per_director.value_counts().sort_index()

# 시각화
plt.figure(figsize=(10, 6))
ax = movie_count_distribution.plot(kind='bar', width=0.8, color='skyblue')
plt.title('Distribution of Movie Count per Director', fontsize=14)
plt.xlabel('Number of Movies per Director', fontsize=12)
plt.ylabel('Number of Direcors', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 막대 위에 개수 표시
for i, count in enumerate(movie_count_distribution):
    plt.text(i, count + 0.5, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.show()


모든 감독이 최소 2개 이상의 영화에 참여했다.

In [ ]:
# 고유 영화 수 (directors_df)
unique_items_directors_df = directors_df['item'].nunique()
print(f"Unique items (directors_df): {unique_items_directors_df}")

# 고유 영화 수 (training)
unique_items_train_df = train_df['item'].nunique()
print(f"Unique items (training): {unique_items_train_df}")

# 영화별 감독 수
items_counts = directors_df['item'].value_counts()
print(director_counts.head(10))  # 상위 10명

모든 영화에 감독 정보가 있는 것은 아님

In [ ]:
# 영화별 감독 수 계산
director_count_per_movie = directors_df['item'].value_counts()  # 영화별 참여한 감독 수

# 감독 수의 분포 계산
director_count_distribution = director_count_per_movie.value_counts().sort_index()

# 시각화
plt.figure(figsize=(10, 6))
ax = director_count_distribution.plot(kind='bar', width=0.8, color='skyblue')
plt.title('Distribution of Director Count per Movie', fontsize=14)
plt.xlabel('Number of Directors per Movie', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 막대 위에 개수 표시
for i, count in enumerate(director_count_distribution):
    plt.text(i, count + 0.5, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.show()


대부분의 영화는 감독이 한 명이다.

In [ ]:
directors_df[directors_df['item'] == 50912]

In [ ]:
titles_df[titles_df['item'] == 50912]

검색하니 앤솔로지 영화였다.

# genres_df EDA

In [ ]:
# 기본 정보 확인
print(genres_df.info())
print(genres_df.head())

# 결측치 확인
print(genres_df.isnull().sum())

# 중복 행 확인
print(genres_df[genres_df.duplicated()])

In [ ]:
# 장르별 영화 개수
genre_counts = genres_df['genre'].value_counts()

# 상위 10개 장르 확인
print(genre_counts.head(10))


장르의 조합의 빈도

In [ ]:
from collections import Counter

# 영화별 장르 리스트 생성
movie_genres = genres_df.groupby('item')['genre'].apply(list)

# 장르 리스트를 정렬된 튜플로 변환 (조합 비교를 위해)
sorted_genre_combinations = movie_genres.apply(lambda x: tuple(sorted(x)))

# 각 조합의 빈도 계산
genre_combination_counts = Counter(sorted_genre_combinations)

# 가장 많은 장르 조합 확인
most_common_combinations = genre_combination_counts.most_common(20)  # 상위 10개
print("Most common genre combinations:")
for combination, count in most_common_combinations:
    print(f"Genres: {combination}, Count: {count}")


2개의 장르 조합 빈도

In [ ]:
from itertools import combinations
from collections import Counter

# 영화별 장르 리스트 생성
movie_genres = genres_df.groupby('item')['genre'].apply(list)

# 모든 영화에서 2개의 장르 조합 생성
genre_combinations = []

for genres in movie_genres:
    if len(genres) >= 2:  # 장르가 2개 이상인 경우만 조합 생성
        genre_combinations.extend(combinations(genres, 2))

# 조합을 정렬하여 중복 제거
sorted_genre_combinations = [tuple(sorted(comb)) for comb in genre_combinations]

# 조합 빈도 계산
combination_counts = Counter(sorted_genre_combinations)

# 상위 10개 조합 출력
print("Top 10 Genre Combinations:")
for combination, count in combination_counts.most_common(10):
    print(f"Combination: {combination}, Count: {count}")


In [ ]:
# 시각화
plt.figure(figsize=(10, 6))
genre_counts.plot(kind='bar', color='skyblue')
plt.title('Number of Movies by Genre', fontsize=14)
plt.xlabel('Genre', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


한 영화의 장르의 개수 분포

In [ ]:
# 영화별 장르 수 계산
genres_per_movie = genres_df.groupby('item')['genre'].count()

# 분포 확인
print(genres_per_movie.describe())

# 시각화
plt.figure(figsize=(10, 6))
ax = genres_per_movie.value_counts().sort_index().plot(kind='bar', color='lightgreen')
plt.title('Number of Genres per Movie', fontsize=14)
plt.xlabel('Number of Genres', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 막대 위에 개수 표시
for i, count in enumerate(genres_per_movie.value_counts().sort_index()):
    plt.text(i, count + 0.5, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
genres_per_movie[genres_per_movie == 10]

In [ ]:
titles_df[titles_df['item'] == 81132]

# titles_df EDA

In [ ]:
# 기본 정보 확인
print(titles_df.info())
print(titles_df.head())

# 결측치 확인
print(titles_df.isnull().sum())

# 중복 행 확인
print(titles_df[titles_df.duplicated()])

# writes_df EDA

In [ ]:
# 기본 정보 확인
print(writers_df.info())
print(writers_df.head())

# 결측치 확인
print(writers_df.isnull().sum())

# 중복 행 확인
print(writers_df[writers_df.duplicated()])

In [ ]:
# 고유 작가 수
unique_writers = writers_df['writer'].nunique()
print(f"Unique writers: {unique_writers}")

# 작가별 빈도
writer_counts = writers_df['writer'].value_counts()
print(writer_counts.head(10))  # 상위 10명

In [ ]:
# 작가 별 영화 수 계산
movie_count_per_writer = writers_df['writer'].value_counts()  # 작가별 참여한 영화 수

# 영화 수의 분포 계산
movie_count_distribution = movie_count_per_writer.value_counts().sort_index()

# 시각화
plt.figure(figsize=(10, 6))
ax = movie_count_distribution.plot(kind='bar', width=0.8, color='skyblue')
plt.title('Distribution of Movie Count per Writer', fontsize=14)
plt.xlabel('Number of Movies per Writer', fontsize=12)
plt.ylabel('Number of Writers', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 막대 위에 개수 표시
for i, count in enumerate(movie_count_distribution):
    plt.text(i, count + 0.5, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.show()


모든 작가가 최소 2개 이상의 영화에 참여했다.

In [ ]:
# 고유 영화 수 (writers_df)
unique_items_writers_df = writers_df['item'].nunique()
print(f"Unique items (writers_df): {unique_items_writers_df}")

# 고유 영화 수 (training)
unique_items_train_df = train_df['item'].nunique()
print(f"Unique items (training): {unique_items_train_df}")

# 영화별 작가 수
items_counts = writers_df['item'].value_counts()
print(writer_counts.head(10))  # 상위 10명

모든 영화에 작가 정보가 있는 것은 아님

In [ ]:
# 영화별 작가 수 계산
writer_count_per_movie = writers_df['item'].value_counts()  # 영화별 참여한 작가 수

# 작가 수의 분포 계산
writer_count_distribution = writer_count_per_movie.value_counts().sort_index()

# 시각화
plt.figure(figsize=(10, 6))
ax = writer_count_distribution.plot(kind='bar', width=0.8, color='skyblue')
plt.title('Distribution of Writer Count per Movie', fontsize=14)
plt.xlabel('Number of Writer per Movie', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 막대 위에 개수 표시
for i, count in enumerate(writer_count_distribution):
    plt.text(i, count + 0.5, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
writer_count_per_movie

In [ ]:
writers_df[writers_df['item'] == 1907]

In [ ]:
titles_df[titles_df['item'] == 1907]

# years_df EDA

In [ ]:
# 기본 정보 확인
print(years_df.info())
print(years_df.head())

# 결측치 확인
print(years_df.isnull().sum())

# 중복 행 확인
print(years_df[years_df.duplicated()])

In [ ]:
year_counts = years_df['year'].value_counts().sort_index()
print(year_counts)

# 시각화
plt.figure(figsize=(12, 6))
year_counts.plot(kind='line', color='skyblue')
plt.title('Number of Movies by Year', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
